<a href="https://colab.research.google.com/github/r-kovalch/acter-ner/blob/main/notebooks/acter-gliner-small-en.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Cloning into 'acter-ner'...
remote: Enumerating objects: 263, done.
remote: Counting objects: 100% (32/32), done.
remote: Compressing objects: 100% (27/27), done.
remote: Total 263 (delta 11), reused 7 (delta 5), pack-reused 231 (from 1)
Receiving objects: 100% (263/263), 2.43 MiB | 3.50 MiB/s, done.
Resolving deltas: 100% (165/165), done.


In [2]:
!git clone https://github.com/AylaRT/ACTER

Cloning into 'ACTER'...
remote: Enumerating objects: 5448, done.
remote: Counting objects: 100% (5448/5448), done.
remote: Compressing objects: 100% (3304/3304), done.
remote: Total 5448 (delta 2684), reused 4893 (delta 2132), pack-reused 0 (from 0)
Receiving objects: 100% (5448/5448), 11.90 MiB | 10.19 MiB/s, done.
Resolving deltas: 100% (2684/2684), done.


In [3]:
%cd /content/ACTER

/content/ACTER


In [4]:
%cd /content/acter-ner/term_extractor

/content/acter-ner/term_extractor


In [5]:
ls

combine_corpora-en-fr-nl.sh  dataset_processor.py        train_full.tsv
combine_corpora-en-fr.sh     output/                     train_model.py
combine_corpora_gliner.sh    preprocess_acter_gliner.py  train_spacy_model.py
combine_corpora.sh           preprocess_acter.py         Untitled.ipynb
configs/                     test_full.tsv


In [6]:
!bash combine_corpora_gliner.sh

In [ ]:
!pip install -U "gliner>=0.2.19" "transformers>=4.51.0" \
               datasets accelerate evaluate --quiet


In [7]:
from datasets import load_dataset
from gliner import GLiNER, TrainingArguments

# 1- Load data ---------------------------------------------------------------
dataset = load_dataset("json", data_files={
    "train": "/content/train.jsonl",
    "validation": "/content/dev.jsonl",
    "test": "/content/test.jsonl"
})

# 2- Pick a base checkpoint --------------------------------------------------
model = GLiNER.from_pretrained("urchade/gliner_medium-v2.1")

# 3- Define label set (you can also compute it programmatically) -------------
label_set = sorted({e["label"] for row in dataset["train"]["entities"]
                               for e in row})

# 4- Training args -----------------------------------------------------------
args = TrainingArguments(
    output_dir="gliner_acter_ft",
    learning_rate=5e-6,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=5,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    fp16=True,                 # works out-of-the-box on Colab A100/T4
)

# 5- Fine-tune ---------------------------------------------------------------
model.fit(
    train_dataset=dataset["train"],
    eval_dataset=dataset["validation"],
    label_list=label_set,
    training_args=args,
)

model.save_pretrained("gliner_acter_ft")


ℹ Auto-detected token-per-line NER format
ℹ Grouping every 1 sentences into a document.
⚠ To generate better training data, you may want to group sentences
into documents with `-n 10`.
✔ Generated output file (11329 documents): output/train_full.spacy
ℹ Auto-detected token-per-line NER format
ℹ Grouping every 1 sentences into a document.
⚠ To generate better training data, you may want to group sentences
into documents with `-n 10`.
✔ Generated output file (2833 documents): output/test_full.spacy


In [ ]:
# from google.colab import runtime
# runtime.unassign()